# Extract selected area from DestinE Climate Twin in Healpix to Zarr

The goal of this notebook is to read 2D variables from GFTS bucket and select a small geographical area and store the results in zarr.


In [1]:
pip install git+https://github.com/xarray-contrib/xdggs.git

  Cloning https://github.com/xarray-contrib/xdggs.git to /tmp/pip-req-build-h8236x1p
  Running command git clone --filter=blob:none --quiet https://github.com/xarray-contrib/xdggs.git /tmp/pip-req-build-h8236x1p
  Resolved https://github.com/xarray-contrib/xdggs.git to commit a570128d39d4c03bf940fe0ade53cae6d0e09c42
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached h3ronpy-0.20.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.1 kB)
Using cached h3ronpy-0.20.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
  Created wheel for xdggs: filename=xdggs-0.0.2.dev26+ga570128-py3-none-any.whl size=16233 sha256=962fa487f757a3f9883d40ceb5419721c772f19a7a856ac068eaea4893900478
  Stored in directory: /tmp/pip-ephem-wheel-cache-i97xylcu/wheels/ea/7e/0a/310dd91a819c3a495c9f3c838ec8216825343c699027d6d1f4
Successfully bui

In [1]:
import xarray as xr
import healpy as hp

import fsspec
import datetime
import os
import s3fs
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import cartopy.crs as ccrs

## 2D variables to process

In [2]:
variables2D = ["avg_sos", "avg_hc300m", "avg_hc700m", "avg_zos"]
years = [2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
months = [6]
models = ["ifs-nemo"]
maxlevels = {}
maxlevels["ifs-nemo"] = 75
maxlevels["icon"] = 72

## Define Geographical area

In [3]:
bbox = {"latitude": [46, 51], "longitude": [-8, -1]}

## Processing

In [4]:
for model in models:
    for var in variables2D:
        for year in years:
            for month in months:
                for p in range(2):
                    start_date = datetime.datetime(
                        year, month, 1
                    ) + p * datetime.timedelta(days=15)
                    end_date = start_date + datetime.timedelta(days=14)
                    uri = (
                        "simplecache::s3://gfts-reference-data/ClimateDT/raw/"
                        + var
                        + "_"
                        + model
                        + "_"
                        + start_date.strftime("%Y%m%d")
                        + "-"
                        + end_date.strftime("%Y%m%d")
                        + ".grib"
                    )
                    print(uri)
                    try:
                        if not os.path.isdir(
                            "small/" + os.path.basename(uri).split(".grib")[0] + ".zarr"
                        ):
                            file = fsspec.open_local(
                                uri,
                                s3={"anon": False},
                                filecache={
                                    "cache_storage": "/home/jovyan/cache_storage"
                                },
                            )
                            dset = xr.open_dataset(file, engine="cfgrib")
                            npix = dset.sizes["values"]
                            nest = True
                            nside = hp.npix2nside(npix)
                            cell_ids = range(0, 12 * nside**2)
                            cell_ids = range(12 * nside**2, 0, -1)
                            dset = dset.assign_coords(
                                {"cell_ids": ("values", cell_ids)}
                            ).swap_dims({"values": "cell_ids"})
                            dset.cell_ids.attrs = {
                                "grid_name": "healpix",
                                "nside": nside,
                                "nest": nest,
                            }
                            dset["longitude"] = ((dset.longitude + 180) % 360) - 180

                            dset.where(
                                (dset.latitude >= bbox["latitude"][0])
                                & (dset.latitude <= bbox["latitude"][1])
                                & (dset.longitude >= bbox["longitude"][0])
                                & (dset.longitude <= bbox["longitude"][1]),
                                drop=True,
                            ).to_zarr(
                                "small/"
                                + os.path.basename(uri).split(".grib")[0]
                                + ".zarr"
                            )
                        else:
                            print("zarr file exists for ", uri)
                    except Exception:
                        print("not processing file ", uri)

simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20200601-20200615.grib
not processing file  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20200601-20200615.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20200616-20200630.grib
not processing file  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20200616-20200630.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20210601-20210615.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20210601-20210615.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20210616-20210630.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20210616-20210630.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-nemo_20220601-20220615.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_sos_ifs-

## Save geographical area cell_ids

In [5]:
var

'avg_zos'

In [6]:
if not os.path.isdir("cell_ids.zarr"):
    dset.isel(time=0).reset_coords("time", drop=True).where(
        (dset.latitude >= bbox["latitude"][0])
        & (dset.latitude <= bbox["latitude"][1])
        & (dset.longitude >= bbox["longitude"][0])
        & (dset.longitude <= bbox["longitude"][1]),
        drop=True,
    ).drop_vars(var).to_zarr("cell_ids.zarr")

## 3D variables to process

In [7]:
variables3D = ["avg_thetao", "avg_so", "avg_von", "avg_uoe", "avg_wo"]

In [8]:
for model in models:
    for var in variables3D:
        for year in years:
            for month in months:
                for p in range(2):
                    start_date = datetime.datetime(
                        year, month, 1
                    ) + p * datetime.timedelta(days=15)
                    end_date = start_date + datetime.timedelta(days=14)
                    uri = (
                        "simplecache::s3://gfts-reference-data/ClimateDT/raw/"
                        + var
                        + "_"
                        + model
                        + "_"
                        + start_date.strftime("%Y%m%d")
                        + "-"
                        + end_date.strftime("%Y%m%d")
                        + ".grib"
                    )
                    print(uri)
                    try:
                        if not os.path.isdir(
                            "small/" + os.path.basename(uri).split(".grib")[0] + ".zarr"
                        ):
                            file = fsspec.open_local(
                                uri,
                                s3={"anon": False},
                                filecache={
                                    "cache_storage": "/home/jovyan/cache_storage"
                                },
                            )
                            dset = xr.open_dataset(
                                file, engine="cfgrib", chunks={"time": 1}
                            )
                            npix = dset.sizes["values"]
                            nest = True
                            nside = hp.npix2nside(npix)
                            cell_ids = range(0, 12 * nside**2)
                            cell_ids = range(12 * nside**2, 0, -1)
                            dset = dset.assign_coords(
                                {"cell_ids": ("values", cell_ids)}
                            ).swap_dims({"values": "cell_ids"})
                            dset.cell_ids.attrs = {
                                "grid_name": "healpix",
                                "nside": nside,
                                "nest": nest,
                            }
                            dset["longitude"] = ((dset.longitude + 180) % 360) - 180
                            dcell_ids = xr.open_dataset("cell_ids.zarr", engine="zarr")
                            dcell_ids = dcell_ids.expand_dims(
                                dim={"time": dset.time.size}
                            )

                            dset.where(
                                dset.cell_ids == dcell_ids.cell_ids, drop=True
                            ).to_zarr(
                                "small/"
                                + os.path.basename(uri).split(".grib")[0]
                                + ".zarr"
                            )
                        else:
                            print("zarr file exists for ", uri)
                    except Exception:
                        print("not processing file ", uri)

simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20200601-20200615.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20200601-20200615.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20200616-20200630.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20200616-20200630.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20210601-20210615.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20210601-20210615.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20210616-20210630.grib
zarr file exists for  simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20210616-20210630.grib
simplecache::s3://gfts-reference-data/ClimateDT/raw/avg_thetao_ifs-nemo_20220601-20220615.grib
zarr file exists for  simplecache::s3://gfts-reference-da

KeyboardInterrupt: 

## Open and consolidate 2D datasets

In [ ]:
list_files = []

In [ ]:
for model in models:
    for var in variables2D:
        for year in years:
            for month in months:
                for p in range(2):
                    start_date = datetime.datetime(
                        year, month, 1
                    ) + p * datetime.timedelta(days=15)
                    end_date = start_date + datetime.timedelta(days=14)
                    zarrfile = (
                        var
                        + "_"
                        + model
                        + "_"
                        + start_date.strftime("%Y%m%d")
                        + "-"
                        + end_date.strftime("%Y%m%d")
                        + ".zarr"
                    )
                    print(zarrfile)

                    if os.path.isdir("small/" + os.path.basename(zarrfile)):
                        list_files.append("small/" + zarrfile)

In [ ]:
dset = xr.open_mfdataset(list_files, engine="zarr")
dset

### Set the path to the remote location

In [ ]:
target2D = fsspec.get_mapper(
    "s3://gfts-reference-data/ClimateDT/bbox_area1/climateDT_2D.zarr",
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    },
)

In [ ]:
dset.chunk("auto").to_zarr(store=target2D, mode="w", consolidated=True, compute=True)

## Open and consolidate 3D datasets

In [ ]:
list_files = []

In [ ]:
for model in models:
    for var in variables3D:
        for year in years:
            for month in months:
                for p in range(2):
                    start_date = datetime.datetime(
                        year, month, 1
                    ) + p * datetime.timedelta(days=15)
                    end_date = start_date + datetime.timedelta(days=14)
                    zarrfile = (
                        var
                        + "_"
                        + model
                        + "_"
                        + start_date.strftime("%Y%m%d")
                        + "-"
                        + end_date.strftime("%Y%m%d")
                        + ".zarr"
                    )
                    print(zarrfile)

                    if os.path.isdir("small/" + os.path.basename(zarrfile)):
                        list_files.append("small/" + zarrfile)

In [ ]:
dset = xr.open_mfdataset(list_files, engine="zarr")
dset

### Set the path to the remote location

In [ ]:
target3D = fsspec.get_mapper(
    "s3://gfts-reference-data/ClimateDT/bbox_area1/climateDT_3D.zarr",
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    },
)

In [ ]:
dset.chunk("auto").to_zarr(store=target3D, mode="w", consolidated=True, compute=True)

## Loading remote zarr

In [9]:
fsg = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    },
)

In [10]:
store = s3fs.S3Map(
    root="s3://gfts-reference-data/ClimateDT/bbox_area1/climateDT_2D.zarr",
    s3=fsg,
    check=False,
)
d2D = xr.open_zarr(store=store, consolidated=True)

In [11]:
d2D

<xarray.Dataset> Size: 33MB
Dimensions:             (time: 291, cell_ids: 7053)
Coordinates:
  * cell_ids            (cell_ids) int64 56kB 9078853 9077897 ... 8960702
    depthBelowSeaLayer  float64 8B ...
    latitude            (cell_ids) float64 56kB dask.array<chunksize=(7053,), meta=np.ndarray>
    longitude           (cell_ids) float64 56kB dask.array<chunksize=(7053,), meta=np.ndarray>
    oceanSurface        float64 8B ...
    step                timedelta64[ns] 8B ...
  * time                (time) datetime64[ns] 2kB 2021-06-01 ... 2030-06-21
    valid_time          (time) datetime64[ns] 2kB dask.array<chunksize=(291,), meta=np.ndarray>
Data variables:
    avg_hc300m          (time, cell_ids) float32 8MB dask.array<chunksize=(8, 7053), meta=np.ndarray>
    avg_hc700m          (time, cell_ids) float32 8MB dask.array<chunksize=(8, 7053), meta=np.ndarray>
    avg_sos             (time, cell_ids) float32 8MB dask.array<chunksize=(8, 7053), meta=np.ndarray>
    avg_zos             (time, cell_ids) float32 8MB dask.array<chunksize=(8, 7053), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-05-23T11:12 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [12]:
store = s3fs.S3Map(
    root="s3://gfts-reference-data/ClimateDT/bbox_area1/climateDT_3D.zarr",
    s3=fsg,
    check=False,
)
d3D = xr.open_zarr(store=store, consolidated=True)
d3D

<xarray.Dataset> Size: 635MB
Dimensions:          (time: 150, oceanModelLayer: 75, cell_ids: 7053)
Coordinates:
  * cell_ids         (cell_ids) int64 56kB 9078853 9077897 ... 8960703 8960702
    latitude         (cell_ids) float64 56kB dask.array<chunksize=(7053,), meta=np.ndarray>
    longitude        (cell_ids) float64 56kB dask.array<chunksize=(7053,), meta=np.ndarray>
  * oceanModelLayer  (oceanModelLayer) float64 600B 1.0 2.0 3.0 ... 74.0 75.0
    oceanSurface     float64 8B ...
    step             timedelta64[ns] 8B ...
  * time             (time) datetime64[ns] 1kB 2020-06-01 ... 2024-06-30
Data variables:
    avg_so           (time, oceanModelLayer, cell_ids) float32 317MB dask.array<chunksize=(1, 75, 7053), meta=np.ndarray>
    avg_thetao       (time, oceanModelLayer, cell_ids) float32 317MB dask.array<chunksize=(1, 75, 7053), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-05-23T12:47 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

## Basic visualization

In [13]:
# hvplot.xarray bugs with 1D index.
# workaround, is put that in pandas.
# but then when putting it into pandas, it bugs thus have to do reset_index before plotting....
#
df = d3D.isel(time=0, oceanModelLayer=0).reset_index("cell_ids").to_dataframe()
df[df.avg_thetao.notna()].hvplot.scatter(
    x="longitude",
    y="latitude",
    c="avg_thetao",
    s=5,
    geo=True,
    global_extent=True,
    frame_height=450,  # , tiles=True
    projection=ccrs.Orthographic(0, 30),
    # , marker='h', size=20
    coastline=True,
)

:Overlay
   .Scatter.I   :Scatter   [longitude]   (latitude,avg_thetao)
   .Coastline.I :Feature   [Longitude,Latitude]

In [14]:
d2D.avg_zos.isel(cell_ids=1000).to_dataframe()["avg_zos"].hvplot()

:Curve   [time]   (avg_zos)